## Βήμα 1,2.  Συλλογή δεδομένων και Προεπεξεργασία κειμένου (Text Processing): 

### libraries του κώδικα :

In [5]:
import requests
import json
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

stop_words = set(word.lower() for word in stopwords.words("english"))
lemmatizer = WordNetLemmatizer()


### Text Processing ΑΛΓΌΡΙΘΜΟΣ :

Το βήμα αυτό εστιάζει στην προεπεξεργασία του συνόλου δεδομένων που ανακτήσαμε από τις ιστοσελίδες στο πρώτο βήμα. Η προεπεξεργασία περιέχει πεζοποίηση των χαρακτήρων, tokenization, δηλαδή διάσπαση του κειμένου σε μεμονωμένους όρους, stemming/lemmatization, δηλαδή αποκοπή των καταλήξεων από κάθε λέξη και προσδιορισμό της κάθε μιας από το λήμμα της, αφαίρεση stop-word, δηλαδή λέξεων που είναι αρκετά κοινές και αφαίρεση ειδικών χαρακτήρων όπως σημείων στίξης

In [8]:
#για το Text Processing 
def tokenized_text(text):   
    tokens = nltk.word_tokenize(text)  # Tokenization
    tokens = [word.lower() for word in tokens if word.lower() not in stop_words]  # Αφαίρεση stop word
    tokens = [word for word in tokens if word not in string.punctuation] # Αφαίρεση σημείων στίξης
    tokens = [lemmatizer.lemmatize(word) for word in tokens] #Lemmatization
    unique_tokens = list(set(tokens))
    return unique_tokens

### WEB CRAWLER ΑΛΓΌΡΙΘΜΟΣ ΚΑΙ ΑΠΟΘΗΚΕΥΣΗ ΣΤΟ JASON :

Το σύνολο δεδομένων που ανακτάται προκύπτει μέσω Web crawler που υλοποιήσαμε με την βοηθεια της BeautifulSoup. Η συλλογή των δεδομένων ξεκινάει από ένα URL της Wikipedia και στη συνέχεια το crawler επισκέπτεται τα υπόλοιπα URL που θα αντικρίσει στο αρχικό link. Ο αριθμός των links που θα επισκεφθεί καθορίζεται από την μεταβλητή max_urls. Για την αποθήκευση των δεδομένων σε μορφή Json υλοποιήσαμε αντίστοιχη συνάρτηση.

In [13]:

#wed crawler algo
def scrape_wikipedia(start_url, max_urls=40, visited=None, scr_c=0):
    if visited is None:
        visited = set() 

    if start_url in visited or scr_c >= max_urls:
        return [], scr_c

    visited.add(start_url)
    print(f"Scraping: {start_url}")
    url_open = requests.get(start_url)
    soup = BeautifulSoup(url_open.content, 'html.parser')
    page_info = []

    # Βρίσκει τον τίτλο της σελίδας
    title = soup.find('h1', {'id': 'firstHeading'}) 
    if title:
        title = title.text.strip()
    else:
        title = "No Title Found"

    # Παίρνει όλο το κείμενο 
    content = soup.find_all(['p', 'li'])  
    full_text = " ".join([element.text.strip() for element in content])

    # tokenize το πλήρες κείμενο
    processed_text = tokenized_text(full_text)

    # Αποθήκευση δεδομένων της σελίδας
    page_info.append({
        "url": start_url,
        "title": title,
        "full_text": full_text,  # Αποθηκεύει το πλήρες κείμενο
        "processed_text": processed_text
    })

    scr_c += 1  # Αύξηση μετρητή scrape για το πόσα έχει κανει scrape 

    # Βρείτε και scrape άλλους συνδέσμους Wikipedia
    for link in soup.find_all('a', href=True):
        if scr_c >= max_urls:
            break

        href = link['href']
        if href.startswith("/wiki/") and not re.search(r':', href):  #για να αγνοεί τα special links
            full_url = "https://en.wikipedia.org" + href
            if full_url not in visited:
                additional_data, scr_c = scrape_wikipedia(
                    full_url, max_urls, visited, scr_c
                )
                page_info.extend(additional_data)

    return page_info, scr_c

# για το βημα 1ο η αποθήκεση στο jason 
def save_to_json(file_name, data):
    with open(file_name, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)
    print(f"Αποθήκευση σε JSON: {file_name}")

# MAIN 
if __name__ == "__main__":
    start_url = input("Δώστε το URL της σελίδας Wikipedia: ")
    max_urls = 15

    # Βήμα 1: Συλλογή δεδομένων
    scraped_data, _ = scrape_wikipedia(start_url, max_urls)
    save_to_json("wikipedia_data.json", scraped_data)  # Αποθήκευση δεδομένων σε json



Δώστε το URL της σελίδας Wikipedia:  https://en.wikipedia.org/wiki/PC


Scraping: https://en.wikipedia.org/wiki/PC
Scraping: https://en.wikipedia.org/wiki/Main_Page
Scraping: https://en.wikipedia.org/wiki/Wikipedia
Scraping: https://en.wikipedia.org/wiki/English_Wikipedia
Scraping: https://en.wikipedia.org/wiki/Home_page
Scraping: https://en.wikipedia.org/wiki/Home_screen
Scraping: https://en.wikipedia.org/wiki/Smartphone_patent_wars
Scraping: https://en.wikipedia.org/wiki/Smartphone
Scraping: https://en.wikipedia.org/wiki/Smartphone_(disambiguation)
Scraping: https://en.wikipedia.org/wiki/This_Man_(Cory_Marks_album)
Scraping: https://en.wikipedia.org/wiki/This_Man_(album)
Scraping: https://en.wikipedia.org/wiki/This_Man_(disambiguation)
Scraping: https://en.wikipedia.org/wiki/This_Man
Scraping: https://en.wikipedia.org/wiki/Conceptual_art
Scraping: https://en.wikipedia.org/wiki/Concept_art
Αποθήκευση σε JSON: wikipedia_data.json


urls που έχει επισκεφτεί το web crawler  

## Βήμα 3. Ευρετήριο (Indexing): 

Το ανεστραμμένο ευρετήριο είναι μια δομή δεδομένων που καταχωρεί για κάθε λέξη τα έγγραφα στα οποία αυτή η λέξη εμφανίζεται. Η διαδικασία αυτή πραγματοποιείται μέσω της συλλογής των tokens κάθε εγγράφου και της καταχώρησης των εγγράφων που περιέχουν κάθε λέξη στο ευρετήριο.

In [15]:
#για το Indexing

def inverted_index(data):
    inv_i = {} # Αρχικοποίηση του ανεστραμμένου ευρετηρίο

    for i, page in enumerate(data):
        doc_id = i + 1 # για να βάλουμε αριθμό σε κάθε έγγραφο 
        for word in page["processed_text"]:
            if word not in inv_i:
                inv_i[word] = set()
            inv_i[word].add(doc_id)

    # Μετατροπή συνόλων σε λίστες μετά τη δημιουργία του ανεστραμμένου
    for word in inv_i:
        inv_i[word] = list(inv_i[word])

    return inv_i

# MAIN 
if __name__ == "__main__":
    # Βήμα 3: Ευρετήριο
    inv_i = inverted_index(scraped_data)
    save_to_json("inverted_index.json", inv_i)  # Αποθήκευση του inverted index

Αποθήκευση σε JSON: inverted_index.json


## Βήμα 4. Μηχανή αναζήτησης (Search Engine):

## Βήμα 4α Επεξεργασία ερωτήματος (Query Processing): 

Το βήμα αυτό αποσκοπεί στην ανάπτυξη μεθόδου η οποία αφού λάβει και αναλύσει ένα ερώτημα του χρήστη, θα ανακτά έγγραφα σχετικά με το ερώτημα μέσω του ανεστραμμένου ευρετηρίου. Η αναζήτηση εγγράφων γίνεται με παράθεση ενός ή περισσότερων όρων(λέξεων), οι οποίοι πρώτα γίνονται tokenized, είτε μεμονωμένα είτε με χρήση Boolean τελεστών(and, or, not).

In [23]:
#για το boolean retrieval του 4α 
def boolean_retrieval(query_tokens, inv_i):
    docs = set() # τα έγγραφα που έχουμε πάρει 
    all_docs = set(doc for postings in inv_i.values() for doc in postings)
    results = []
    
    
    for token in query_tokens:
        if token in {"and", "or", "not"}:
            results.append(token)
        else:
            preprocessed_tokens = tokenized_text(token)   # επεξεργασία όρων του query
            if preprocessed_tokens:
                results.append(preprocessed_tokens[0])
    
    for i in range(len(results)):
        token = results[i]
        if token == "and":
            i += 1
            term = results[i]
            doc_with_term = set(inv_i.get(term, []))
            docs = docs & doc_with_term if docs else doc_with_term
        elif token == "or":
            i += 1
            term = results[i]
            doc_with_term = set(inv_i.get(term, []))
            docs = docs | doc_with_term if docs else doc_with_term
        elif token == "not":
            i += 1
            term = results[i]
            doc_with_term = set(inv_i.get(term, []))
            docs = docs - doc_with_term if docs else all_docs - doc_with_term 
        else:  # απλός όρος
            term = token
            doc_with_term = set(inv_i.get(term, []))
            docs = docs | doc_with_term
    return docs   


# MAIN 
if __name__ == "__main__":
       # Βήμα 4: Επεξεργασία ερωτήματος και κατάταξη αποτελεσμάτων
    query = input("Δώστε τους όρους της αναζήτησής σας: ")
    # Boolean Retrieval
    retrieved_docs = boolean_retrieval(tokenized_text(query), inv_i)
    print(f"Retrieved (Boolean Retrieval): {retrieved_docs}")

Δώστε τους όρους της αναζήτησής σας:  pc and war


Retrieved (Boolean Retrieval): {1, 2, 3, 4, 7, 8}


## Βήμα 4β Κατάταξη αποτελεσμάτων (Ranking): 

### BOOLEAN RANKING 

In [33]:
#για το  Vector Space Model
def alg_vector_space_Model(vector_a, vector_b):
    product = np.dot(vector_a, vector_b) # Υπολογισμός εσωτερικού γινομένου
    mag = np.linalg.norm(vector_a) * np.linalg.norm(vector_b) # Υπολογισμός μέτρου
    return product / mag if mag else 0 # Επιστροφή συσχέτισης 


#για το tfidf 
def alg_tfidf_pin(data):
    corpus = [" ".join(page["processed_text"]) for page in data]  # Δημιουργία κειμένου 
    vectorizer = TfidfVectorizer()  # Αρχικοποίηση 
    tfidf_pin = vectorizer.fit_transform(corpus)  # Εφαρμογή του TF-IDF 
    feature_names = vectorizer.get_feature_names_out()  # Λήψη των χαρακτηριστικών
    return tfidf_pin, feature_names, vectorizer


def alg_tfidf_ranking(query, tfidf_pin, characters, vector):
    query_vector = vector.transform([" ".join(tokenized_text(query))])  
    scores = (tfidf_pin @ query_vector.T).toarray().flatten()  # Υπολογισμός βαθμολογίας για το ποσο ίδια είναι
    ranked_docs = np.argsort(-scores) #ταξινόμηση σε  φθίνουσα σειρά
    return ranked_docs, scores

#Για το boolean ranking

def alg_boolean_ranking(query_tokens, inv_i, tfidf_pin, characters, vector):
    # Αρχικοποίηση ενός κενού συνόλου που θα περιέχει τα έγγραφα που ταιριάζουν με το ερώτημα.
    docs = set()  

    # Δημιουργία συνόλου με όλα τα διαθέσιμα έγγραφα από το ανεστραμμένο ευρετήριο.
    all_docs = set(doc for postings in inv_i.values() for doc in postings)

    # Λίστα που θα περιέχει τους όρους του ερωτήματος και τους λογικούς τελεστές.
    results = []

    # Λεξικό για να αποθηκευτούν οι βαθμολογίες των εγγράφων.
    doc_scores = {}

    # **Ανάλυση του ερωτήματος και εφαρμογή λογικών πράξεων**
    for token in query_tokens:
        # Αν ο όρος είναι λογικός τελεστής ("and", "or", "not"), τον προσθέτουμε ως έχει στη λίστα.
        if token in {"and", "or", "not"}:
            results.append(token)
        else:
            # Επεξεργασία των όρων του ερωτήματος (lemmatization, αφαίρεση stop words κ.λπ.).
            preprocessed_tokens = tokenized_text(token)
            if preprocessed_tokens:
                # Προσθέτουμε τον επεξεργασμένο όρο στη λίστα.
                results.append(preprocessed_tokens[0])

    # λογικές πράξεις 
    for i in range(len(results)):
        token = results[i]
        if token == "and":  #  "AND".
            i += 1
            term = results[i]
            # Βρίσκουμε τα έγγραφα που εχουν τον όρο
            doc_with_term = set(inv_i.get(term, []))
            
            docs = docs & doc_with_term if docs else doc_with_term
        elif token == "or":  # "OR".
            i += 1
            term = results[i]
            doc_with_term = set(inv_i.get(term, []))
            # Επιστρέφουμε την ένωση των εγγράφων που έχουν συλλεχθεί με αυτά που περιέχουν τον όρο.
            docs = docs | doc_with_term if docs else doc_with_term
        elif token == "not":  # Λογικός τελεστής "NOT".
            i += 1
            term = results[i]
            doc_with_term = set(inv_i.get(term, []))
            # Αφαιρούμε από τα έγγραφα αυτά που περιέχουν τον όρο
            docs = docs - doc_with_term if docs else all_docs - doc_with_term
        else:  # Απλός όρος 
            term = token
            doc_with_term = set(inv_i.get(term, []))
            docs = docs | doc_with_term

    #Υπολογισμός βαθμολογιών 
    for doc_id in docs:
        score = 0
        for term in query_tokens:
            # Ελέγχουμε αν ο όρος υπάρχει 
            if term in characters:
                term_index = characters.tolist().index(term)  # Εντοπισμός του όρου
                # Προσθέτουμε τη βαθμολογία
                score += tfidf_pin[doc_id - 1, term_index]  
        doc_scores[doc_id] = score

    # ταξινόμηση
    ranked_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
    ranked_doc_ids = [doc_id for doc_id, _ in ranked_docs]
    return ranked_doc_ids, doc_scores


# MAIN 
if __name__ == "__main__":
    
    tfidf_pin, feature_names, vectorizer = alg_tfidf_pin(scraped_data)
    
    # Βήμα 4: Επεξεργασία ερωτήματος και κατάταξη αποτελεσμάτων
    query = input("Δώστε τους όρους της αναζήτησής σας: ")
    algo_choice = input("Επιλέξτε μέθοδο ανάκτησης(boolean/vsm): ").strip().lower()
    # Boolean Ranking
    if algo_choice == "boolean":
        
        # Boolean Retrieval
        retrieved_docs = boolean_retrieval(tokenized_text(query), inv_i)
        print(f"Retrieved (Boolean Retrieval): {retrieved_docs}")
        ranked_docs, doc_scores = alg_boolean_ranking(tokenized_text(query), inv_i, tfidf_pin, feature_names, vectorizer)
        print("\nκατάταξη αποτελεσμάτων (Boolean Ranking):")
        for rank, doc_id in enumerate(ranked_docs[:15], start=1):  
            print(f"{rank}. Document {doc_id} (Score: {doc_scores[doc_id]:.4f})")
            print(f"   Title: {scraped_data[doc_id - 1]['title']}")
            print(f"   URL: {scraped_data[doc_id - 1]['url']}")
     # VSM/TF-IDF Ranking
    elif algo_choice == "vsm":
        # VSM/TF-IDF Ranking
        vsm_ranked_docs, scores = alg_tfidf_ranking(query, tfidf_pin, feature_names, vectorizer)
        print("\nκατάταξη αποτελεσμάτων (VSM):")
        for rank, doc_id in enumerate(vsm_ranked_docs[:15], start=1): 
            print(f"{rank}. Document {doc_id + 1} (Score: {scores[doc_id]:.4f})")
            print(f"   Title: {scraped_data[doc_id]['title']}")
            print(f"   URL: {scraped_data[doc_id]['url']}")

Δώστε τους όρους της αναζήτησής σας:  pc or war
Επιλέξτε μέθοδο ανάκτησης(boolean/vsm):  boolean


Retrieved (Boolean Retrieval): {1, 2, 3, 4, 7, 8}

κατάταξη αποτελεσμάτων (Boolean Ranking):
1. Document 8 (Score: 0.0456)
   Title: Smartphone
   URL: https://en.wikipedia.org/wiki/Smartphone
2. Document 1 (Score: 0.0359)
   Title: PC
   URL: https://en.wikipedia.org/wiki/PC
3. Document 2 (Score: 0.0357)
   Title: Main Page
   URL: https://en.wikipedia.org/wiki/Main_Page
4. Document 7 (Score: 0.0345)
   Title: Smartphone patent wars
   URL: https://en.wikipedia.org/wiki/Smartphone_patent_wars
5. Document 4 (Score: 0.0226)
   Title: English Wikipedia
   URL: https://en.wikipedia.org/wiki/English_Wikipedia
6. Document 3 (Score: 0.0131)
   Title: Wikipedia
   URL: https://en.wikipedia.org/wiki/Wikipedia


Αποτέλεσμα boolean ranking 

### VSM RANKING

Vector Space Model, στην οποία κάθε έγγραφο και το ερώτημα αναπαρίσταται ως διάνυσμα στον n-διάστατο χώρο λέξεων. Η αναπαράσταση αυτή χρησιμοποιεί την τιμή του TF-IDF για κάθε λέξη, η οποία συνυπολογίζει τη συχνότητα εμφάνισης της λέξης σε κάθε έγγραφο σε σχέση με τη γενική συχνότητά της σε όλα τα έγγραφα. Η κατάταξη των εγγράφων γίνεται με βάση την ομοιότητα μεταξύ του διανύσματος του ερωτήματος και των διανυσμάτων των εγγράφων, χρησιμοποιώντας το συντελεστή συσχέτισης.

In [36]:
#για το  Vector Space Model
def alg_vector_space_Model(vector_a, vector_b):
    product = np.dot(vector_a, vector_b) # Υπολογισμός εσωτερικού γινομένου
    mag = np.linalg.norm(vector_a) * np.linalg.norm(vector_b) # Υπολογισμός μέτρου
    return product / mag if mag else 0 # Επιστροφή συσχέτισης 


#για το tfidf 
def alg_tfidf_pin(data):
    corpus = [" ".join(page["processed_text"]) for page in data]  # Δημιουργία κειμένου 
    vectorizer = TfidfVectorizer()  # Αρχικοποίηση 
    tfidf_pin = vectorizer.fit_transform(corpus)  # Εφαρμογή του TF-IDF 
    feature_names = vectorizer.get_feature_names_out()  # Λήψη των χαρακτηριστικών
    return tfidf_pin, feature_names, vectorizer


def alg_tfidf_ranking(query, tfidf_pin, characters, vector):
    query_vector = vector.transform([" ".join(tokenized_text(query))])  
    scores = (tfidf_pin @ query_vector.T).toarray().flatten()  # Υπολογισμός βαθμολογίας για το ποσο ίδια είναι
    ranked_docs = np.argsort(-scores) #ταξινόμηση σε  φθίνουσα σειρά
    return ranked_docs, scores

#Για το boolean ranking

def alg_boolean_ranking(query_tokens, inv_i, tfidf_pin, characters, vector):
    # Αρχικοποίηση ενός κενού συνόλου που θα περιέχει τα έγγραφα που ταιριάζουν με το ερώτημα.
    docs = set()  

    # Δημιουργία συνόλου με όλα τα διαθέσιμα έγγραφα από το ανεστραμμένο ευρετήριο.
    all_docs = set(doc for postings in inv_i.values() for doc in postings)

    # Λίστα που θα περιέχει τους όρους του ερωτήματος και τους λογικούς τελεστές.
    results = []

    # Λεξικό για να αποθηκευτούν οι βαθμολογίες των εγγράφων.
    doc_scores = {}

    # **Ανάλυση του ερωτήματος και εφαρμογή λογικών πράξεων**
    for token in query_tokens:
        # Αν ο όρος είναι λογικός τελεστής ("and", "or", "not"), τον προσθέτουμε ως έχει στη λίστα.
        if token in {"and", "or", "not"}:
            results.append(token)
        else:
            # Επεξεργασία των όρων του ερωτήματος (lemmatization, αφαίρεση stop words κ.λπ.).
            preprocessed_tokens = tokenized_text(token)
            if preprocessed_tokens:
                # Προσθέτουμε τον επεξεργασμένο όρο στη λίστα.
                results.append(preprocessed_tokens[0])

    # λογικές πράξεις 
    for i in range(len(results)):
        token = results[i]
        if token == "and":  #  "AND".
            i += 1
            term = results[i]
            # Βρίσκουμε τα έγγραφα που εχουν τον όρο
            doc_with_term = set(inv_i.get(term, []))
            
            docs = docs & doc_with_term if docs else doc_with_term
        elif token == "or":  # "OR".
            i += 1
            term = results[i]
            doc_with_term = set(inv_i.get(term, []))
            # Επιστρέφουμε την ένωση των εγγράφων που έχουν συλλεχθεί με αυτά που περιέχουν τον όρο.
            docs = docs | doc_with_term if docs else doc_with_term
        elif token == "not":  # Λογικός τελεστής "NOT".
            i += 1
            term = results[i]
            doc_with_term = set(inv_i.get(term, []))
            # Αφαιρούμε από τα έγγραφα αυτά που περιέχουν τον όρο
            docs = docs - doc_with_term if docs else all_docs - doc_with_term
        else:  # Απλός όρος 
            term = token
            doc_with_term = set(inv_i.get(term, []))
            docs = docs | doc_with_term

    #Υπολογισμός βαθμολογιών 
    for doc_id in docs:
        score = 0
        for term in query_tokens:
            # Ελέγχουμε αν ο όρος υπάρχει 
            if term in characters:
                term_index = characters.tolist().index(term)  # Εντοπισμός του όρου
                # Προσθέτουμε τη βαθμολογία
                score += tfidf_pin[doc_id - 1, term_index]  
        doc_scores[doc_id] = score

    # ταξινόμηση
    ranked_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
    ranked_doc_ids = [doc_id for doc_id, _ in ranked_docs]
    return ranked_doc_ids, doc_scores


# MAIN 
if __name__ == "__main__":
    
    tfidf_pin, feature_names, vectorizer = alg_tfidf_pin(scraped_data)
    
    # Βήμα 4: Επεξεργασία ερωτήματος και κατάταξη αποτελεσμάτων
    query = input("Δώστε τους όρους της αναζήτησής σας: ")
    algo_choice = input("Επιλέξτε μέθοδο ανάκτησης(boolean/vsm): ").strip().lower()
    # Boolean Ranking
    if algo_choice == "boolean":
        
        # Boolean Retrieval
        retrieved_docs = boolean_retrieval(tokenized_text(query), inv_i)
        print(f"Retrieved (Boolean Retrieval): {retrieved_docs}")
        ranked_docs, doc_scores = alg_boolean_ranking(tokenized_text(query), inv_i, tfidf_pin, feature_names, vectorizer)
        print("\nκατάταξη αποτελεσμάτων (Boolean Ranking):")
        for rank, doc_id in enumerate(ranked_docs[:15], start=1):  
            print(f"{rank}. Document {doc_id} (Score: {doc_scores[doc_id]:.4f})")
            print(f"   Title: {scraped_data[doc_id - 1]['title']}")
            print(f"   URL: {scraped_data[doc_id - 1]['url']}")
     # VSM/TF-IDF Ranking
    elif algo_choice == "vsm":
        # VSM/TF-IDF Ranking
        vsm_ranked_docs, scores = alg_tfidf_ranking(query, tfidf_pin, feature_names, vectorizer)
        print("\nκατάταξη αποτελεσμάτων (VSM):")
        for rank, doc_id in enumerate(vsm_ranked_docs[:15], start=1): 
            print(f"{rank}. Document {doc_id + 1} (Score: {scores[doc_id]:.4f})")
            print(f"   Title: {scraped_data[doc_id]['title']}")
            print(f"   URL: {scraped_data[doc_id]['url']}")

Δώστε τους όρους της αναζήτησής σας:  pc or war
Επιλέξτε μέθοδο ανάκτησης(boolean/vsm):  vsm



κατάταξη αποτελεσμάτων (VSM):
1. Document 8 (Score: 0.0331)
   Title: Smartphone
   URL: https://en.wikipedia.org/wiki/Smartphone
2. Document 1 (Score: 0.0265)
   Title: PC
   URL: https://en.wikipedia.org/wiki/PC
3. Document 7 (Score: 0.0244)
   Title: Smartphone patent wars
   URL: https://en.wikipedia.org/wiki/Smartphone_patent_wars
4. Document 2 (Score: 0.0241)
   Title: Main Page
   URL: https://en.wikipedia.org/wiki/Main_Page
5. Document 4 (Score: 0.0152)
   Title: English Wikipedia
   URL: https://en.wikipedia.org/wiki/English_Wikipedia
6. Document 3 (Score: 0.0092)
   Title: Wikipedia
   URL: https://en.wikipedia.org/wiki/Wikipedia
7. Document 5 (Score: 0.0000)
   Title: Home page
   URL: https://en.wikipedia.org/wiki/Home_page
8. Document 6 (Score: 0.0000)
   Title: Home screen
   URL: https://en.wikipedia.org/wiki/Home_screen
9. Document 9 (Score: 0.0000)
   Title: Smartphone (disambiguation)
   URL: https://en.wikipedia.org/wiki/Smartphone_(disambiguation)
10. Document 10 (

Αποτέλεσμα vsm

### Βήμα 5. Αξιολόγηση συστήματος: 

Το βήμα αυτό ασχολείται με την αξιολόγηση του συστήματος ανάκτησής μας. Οι αποδόσεις μετρούνται με ακρίβεια, ανάκτηση, f1-score και μέση ακρίβεια.

In [47]:
# Βήμα 5ο Αξιολόγηση συστήματος
    
def evaluation(retrieved_docs, relevant_docs):
    # Μετατροπή σε set
    retrieved_set = set(retrieved_docs)
    relevant_set = set(relevant_docs)

    true_positives = len(retrieved_set & relevant_set) # Τομή ανακτηθέντων με σχετικών εγγράφων
    false_positives = len(retrieved_set - relevant_set) # Ανεκτηθέντα αλλά μη σχετικά
    false_negatives = len(relevant_set - retrieved_set) # Σχετικά αλλά μη ανεκτηθέντα

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1

def MAP(retrieved, queries):
    avg = []
    for i, (query_text, relevant_docs) in enumerate(queries):
        # Μετατροπή σε set
        relevant_docs = set(relevant_docs)
        retrieved_docs = set(retrieved[i])

        precision_sum = 0
        relevant_found = 0
        # Υπολογισμός ακρίβειας 
        for rank, doc in enumerate(retrieved_docs, start = 1):
            if doc in relevant_docs:
                relevant_found += 1
                precision_sum += relevant_found / rank
        avg.append(precision_sum / len(relevant_docs) if relevant_docs else 0)
    return sum(avg) / len(avg)      
# MAIN 
if __name__ == "__main__":
  # Βήμα 5: Αξιολόγηση συστήματος
    retrieved = []
    
    queries = [
        ("pc and home", [1, 3, 7, 8]),
        ("data or wiki", [4, 3]),
        ("data not therefore", [2, 4])
    ]
    
    
    for query_text, relevant_docs in queries:
        #  κατάταξη αποτελεσμάτων με  Boolean retrieval και vsm 
        boolean_ranked_docs, _ = alg_boolean_ranking(tokenized_text(query_text), inv_i, tfidf_pin, feature_names, vectorizer)
        vsm_ranked_docs, _ = alg_tfidf_ranking(query_text, tfidf_pin, feature_names, vectorizer)
        #  κατάταξη αποτελεσμάτων με  Boolean retrieval
        ranked_boolean_filtered = [doc_id for doc_id in boolean_ranked_docs if doc_id in vsm_ranked_docs]
        retrieved.append(ranked_boolean_filtered)
    
    print(f"\n After Ranking: {retrieved}")

    for i, query in enumerate(queries):
        precision, recall, f1 = evaluation(retrieved[i], relevant_docs) 
        print(f"Query {i+1}: Precision = {precision:.2}, Recall = {recall:.2}, F1 = {f1:.2}")
    map_score = MAP(retrieved, queries)
    print(f"Mean Average Precision: {map_score:.2}")



 After Ranking: [[6, 5, 8, 1, 7, 4, 3], [4, 6, 3, 8, 7, 14], [6, 14, 4, 8, 7, 3]]
Query 1: Precision = 0.14, Recall = 0.5, F1 = 0.22
Query 2: Precision = 0.17, Recall = 0.5, F1 = 0.25
Query 3: Precision = 0.17, Recall = 0.5, F1 = 0.25
Mean Average Precision: 0.67
